In [1]:
import os
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [22.8 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [947 kB]
Hit:12 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/p

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-24 04:26:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.48MB/s    in 0.2s    

2022-05-24 04:26:49 (5.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_review_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(
    SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"),
    sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

###### Clean and Filter DataFrame to create two tables showing rating breakdowns for products included and not included in the Vine program

In [5]:
# Drop product_id = "B000002O1H", its duplicates are persisting...
df_cleaned = df.filter(df["product_id"] != "B000002O1H")
df_cleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df_cleaned.select(["review_id", "star_rating", "helpful_votes",
                             "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [7]:
# Filter vine_df to retrieve all rows with total_votes >= 20
vine_high_tot_votes = vine_df.filter(vine_df["total_votes"] >= 20)
vine_high_tot_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [8]:
# Filter vine_high_tot_votes to retrieve all rows where helpful_votes / total_votes >= 0.5
vine_majority_helpful = vine_high_tot_votes.filter((vine_high_tot_votes["helpful_votes"] / vine_high_tot_votes["total_votes"]) >= 0.5)
vine_majority_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [9]:
# Filter vine_majority_helpful to retrieve all rows where a vine review was written
# i.e vine == "Y" (paid)
vine_helpful_with_rev = vine_majority_helpful.filter(vine_majority_helpful["vine"] == "Y")
vine_helpful_with_rev.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+



In [10]:
# Filter vine_majority_helpful again, this time to retrieve all rows where a vine review was not written
# i.e vine == "N" (unpaid)
vine_helpful_without_rev = vine_majority_helpful.filter(vine_majority_helpful["vine"] == "N")
vine_helpful_without_rev.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [11]:
# For both vine_helpful_with_rev (paid) and vine_helpful_without_rev (unpaid), determine:
# 1. Number of 5 star reviews
# 2. Percentage of 5 star reviews
from pyspark.sql.functions import count
review_counts_paid_df = vine_helpful_with_rev.groupby("star_rating").\
                                                agg(count("star_rating"))
print("Rating Breakdown (Paid):")
review_counts_paid_df.show()

review_counts_unpaid_df = vine_helpful_without_rev.groupby("star_rating").\
                                                     agg(count("star_rating"))
print("Rating Breakdown (Unpaid):")
review_counts_unpaid_df.show()

Rating Breakdown (Paid):
+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          3|                 3|
|          4|                 4|
+-----------+------------------+

Rating Breakdown (Unpaid):
+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          1|              9157|
|          3|              7897|
|          5|             67580|
|          4|             15997|
|          2|              5348|
+-----------+------------------+



### Summarize Number and Percent of 5 Star Reviews for both Paid and Unpaid

In [12]:
def summarize_reviews(df):
    """
    Function to read results dataframes and return the 5 star rating count and
    percent

    Parameters:
    -----------
    df : pyspark.sql.dataframe.DataFrame
        PySpark data frame with each rating/count

    Returns:
    --------
    (n_total, n_5star, pct_5star) : tuple
        Tuple containing the total number of ratings, number of 5 star
        ratings, and percent of 5 star ratings for the input dataframe
    """
    results_5star = df.filter(df["star_rating"] == "5").collect()
    if results_5star != []:
        n_5star = results_5star[0].asDict()["count(star_rating)"]
    else:
        n_5star = 0
    results_other = df.filter(df["star_rating"] != "5").collect()
    n_other = 0
    for row in results_other:
        n_other += row.asDict()["count(star_rating)"]
    n_total = n_5star + n_other
    pct_5star = n_5star / n_total
    return (n_total, n_5star, pct_5star)

In [13]:
# Create Dataframe for results:
from pyspark.sql.types import StructField, StringType, IntegerType, FloatType, StructType
schema = [StructField("Pay_Status", StringType(), True), StructField("N_Total", IntegerType(), True), StructField("N_5Star", IntegerType(), True), StructField("Pct_5Star", FloatType(), True)]
final = StructType(fields=schema)

results_paid = summarize_reviews(review_counts_paid_df)
results_unpaid = summarize_reviews(review_counts_unpaid_df)
results_df = spark.createDataFrame([
                                    ("Paid", results_paid[0], results_paid[1], results_paid[2] * 100),
                                    ("Unpaid", results_unpaid[0], results_unpaid[1], results_unpaid[2] * 100)
], final)

results_df.show()

+----------+-------+-------+---------+
|Pay_Status|N_Total|N_5Star|Pct_5Star|
+----------+-------+-------+---------+
|      Paid|      7|      0|      0.0|
|    Unpaid| 105979|  67580| 63.76735|
+----------+-------+-------+---------+

